<a href="https://colab.research.google.com/github/rovemin/POOM/blob/main/technology-verification/DogTextAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# sentence-transformers datasets 설치
!pip install sentence-transformers datasets

# 실행에 필요한 패키지 및 라이브러리 불러오기
import math
import logging
from datetime import datetime
import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 1.1 MB/s 
     |████████████████████████████████| 451 kB 31.3 MB/s 
     |████████████████████████████████| 5.5 MB 46.3 MB/s 
     |████████████████████████████████| 1.3 MB 69.6 MB/s 
     |████████████████████████████████| 182 kB 52.7 MB/s 
     |████████████████████████████████| 7.6 MB 21.6 MB/s 
     |████████████████████████████████| 115 kB 60.0 MB/s 
     |████████████████████████████████| 212 kB 26.9 MB/s 
     |████████████████████████████████| 127 kB 48.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=7522bfab7a8cb98b2e98dbfd2a767b26312bb10dab5a6a4757de0532a93f4617
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
  Attempting uninstall: urllib3
    Found existing i

In [ ]:
# Sentence-Transformers 학습시키기

logging.basicConfig(
    format="%(asctime)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    level=logging.INFO,
    handlers=[LoggingHandler()],
)

model_name = "klue/roberta-base"

train_batch_size = 32
num_epochs = 4
model_save_path = "output/training_klue_sts_" + model_name.replace("/", "-") + "-" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

embedding_model = models.Transformer(model_name)

pooler = models.Pooling(
    embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False,
)

model = SentenceTransformer(modules=[embedding_model, pooler])

datasets = load_dataset("klue", "sts")

datasets["train"][0]

testsets = load_dataset("kor_nlu", "sts")

testsets["test"][0]

train_samples = []
dev_samples = []
test_samples = []

# KLUE STS 내 훈련, 검증 데이터 예제 변환
for phase in ["train", "validation"]:
    examples = datasets[phase]

    for example in examples:
        score = float(example["labels"]["label"]) / 5.0  # 0.0 ~ 1.0 스케일로 유사도 정규화

        inp_example = InputExample(
            texts=[example["sentence1"], example["sentence2"]], 
            label=score,
        )

        if phase == "validation":
            dev_samples.append(inp_example)
        else:
            train_samples.append(inp_example)

# KorSTS 내 테스트 데이터 예제 변환
for example in testsets["test"]:
    score = float(example["score"]) / 5.0

    if example["sentence1"] and example["sentence2"]:
        inp_example = InputExample(
            texts=[example["sentence1"], example["sentence2"]],
            label=score,
        )

    test_samples.append(inp_example)

train_dataloader = DataLoader(
    train_samples,
    shuffle=True,
    batch_size=train_batch_size,
)
train_loss = losses.CosineSimilarityLoss(model=model)

evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    dev_samples,
    name="sts-dev",
)

warmup_steps = math.ceil(len(train_dataloader) * num_epochs  * 0.1)  # 10% of train data for warm-up
logging.info(f"Warmup-steps: {warmup_steps}")

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=evaluator,
    epochs=num_epochs,
    evaluation_steps=1000,
    warmup_steps=warmup_steps,
    output_path=model_save_path,
)

model = SentenceTransformer(model_save_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='sts-test')

test_evaluator(model, output_path=model_save_path)

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

...

Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

0.7685737821712351

In [ ]:
docs = [
    "귀가 토끼처럼 쫑긋하고 전체적으로 흰색인데 등쪽은 옅은 갈색털이 섞여있습니다",
    "귀가 꽤 크며, 등에 점이 많음",
    "갈색 털의 푸들이며 털이 짧고 목에 이름표가 붙어져 있습니다.",
    "털이 길지 않고 입 주변이 갈색입니다",
    "귀가 넓고 털은 흰 색이지만 등 안 쪽에 베이지색 털이 자라고 있습니다.",
    "꼬리가 짧고 검은색에 갈색이 섞여있음. 소심한 성격으로 순한 편임",
    "몸집이 작고 겁이 많음 꼬리가 짧음"
]
document_embeddings = model.encode(docs)

In [ ]:
query = "털이 갈색이고 꼬리가 짧습니다."
query_embedding = model.encode(query)

In [ ]:
top_k = min(5, len(docs))

# 입력 문장 - 문장 후보군 간 코사인 유사도 계산 후,
cos_scores = util.pytorch_cos_sim(query_embedding, document_embeddings)[0]

# 코사인 유사도 순으로 `top_k` 개 문장 추출
top_results = torch.topk(cos_scores, k=top_k)

print(f"입력 문장: {query}")
print(f"\n<입력 문장과 유사한 {top_k} 개의 문장>\n")

for i, (score, idx) in enumerate(zip(top_results[0], top_results[1])):
    print(f"{i+1}: {docs[idx]} {'(유사도: {:.4f})'.format(score)}\n")

입력 문장: 털이 갈색이고 꼬리가 짧습니다.

<입력 문장과 유사한 5 개의 문장>

1: 꼬리가 짧고 검은색에 갈색이 섞여있음. 소심한 성격으로 순한 편임 (유사도: 0.8349)

2: 털이 길지 않고 입 주변이 갈색입니다 (유사도: 0.7924)

3: 갈색 털의 푸들이며 털이 짧고 목에 이름표가 붙어져 있습니다. (유사도: 0.7599)

4: 귀가 토끼처럼 쫑긋하고 전체적으로 흰색인데 등쪽은 옅은 갈색털이 섞여있습니다 (유사도: 0.6427)

5: 귀가 넓고 털은 흰 색이지만 등 안 쪽에 베이지색 털이 자라고 있습니다. (유사도: 0.6264)

